<a href="https://colab.research.google.com/github/wsricardo/news-crawler/blob/main/NewsFeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feed News - Database

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
import json

## Download das Notícias do Banco de Dados No Repositório NewsFeed

O presente arquivos não contêm corpo dos artigos jornalisticos de seus respectivos portais por a fim de evitar possíveis implicações de ***Copy Right***.

Preferencialmente baixe o conteúdo localmente para evitar repetidas requisições ao site.

In [3]:
# URL Base de onde são obtidos as noticias (dados armazenados em arquivos json).
urlbase = 'https://raw.githubusercontent.com/wsricardo/newsfeed/main/database/'
#https://raw.githubusercontent.com/wsricardo/newsfeed/main/database/
# List files news links
urlmap = 'https://raw.githubusercontent.com/wsricardo/newsfeed/main/map'


In [4]:
# Criando uma sessão pra requisitar os arquivos. Arquivo 'map' contêm lista dos nomes dos arquivos *.json armazenados em database.
mapnews_session = requests.Session()
# Salva os nomes em news_filws
news_files = mapnews_session.get(urlmap)


In [5]:
# Cria uma lista com os nomes dos arquivos.
news_files_names = news_files.text.split( '\n' )
# Olhando as Duas primeiras. Checando.
news_files_names[:2]

['Noticias-10-5-2023.json', 'Noticias-10-6-2023.json']

Vamos obter os arquivos json contendo todos links e titulos dos artigos.

In [6]:
# Sessão para obter/baixar os dados contidos nos arquivos json.
news_req = requests.Session()

#https://raw.githubusercontent.com/wsricardo/newsfeed/main/database/Noticias-10-5-2023.json
#news1 = json.load( news_req.get( urlbase+news_files_names[0] ).text )

In [7]:
#dfl = pd.DataFrame( news1 )
dfl = []
news_files_names.pop() # última linha estáva vazia quando aplicou a função 'split' na string retornada no arquivomap. O ideal é checar anteriormente.



''

In [8]:
# Obtendo arquidos. São lidos e criados um dataframe para cada em dfl.
for i in news_files_names:
  #print('> i ',i)
  dfl.append( pd.read_json( news_req.get( urlbase+i ).text ) )

In [9]:
# OS dataframes são contenados em um unico dataframe.
df = pd.concat( dfl )

## Limpando os Dados

A função dfclean servirá para limpeza removendo itens duplicados e que não sejam links para artigos (exemplo links para páginas como instagram, youtube, páginas de sessões que não contem artigo sendo referencia para outras páginas.

In [10]:
def dfcleans( df ):

    """
    Remove fields by title in list of names specifieds.
    """

    # Remove lines specified to link of Twitter
    v = [ 'Política',
        'Pop',
        'Economia',
        'Internacional',
        'Viagem & Gastronomia',
        'Nacional',
        'Saúde',
        'Esportes',
        'CNN Plural',
        'Ciência e Tecnologia',
        'Mercado',
        'Facebook',
        'Twitter',
        'Youtube',
        'YouTube',
        'Instagram'
    ]

    df.rename( columns={ 'title': 'Title' }, inplace = True   )
    #dff.rename( columns={ 'href': 'Link' }, inplace = True )



    for i in v:
        df = df.drop( index=df.index[  df['Title'] == i  ] )

    df = df.drop_duplicates( ignore_index = True )
    return df.to_json( orient='records' )

Aqui aplicasse a função de limpeza e criasse um novo dataframe.

In [11]:
df = pd.DataFrame( json.loads( dfcleans( pd.concat( dfl ) ) ) )

Na tabela exibida abaixo clique no icone da "caneta mágica" ao lado direito da tabela dos dados para ativar a tabela interativa permitindo acessar algumas funcionalidades de filtragem e pesquisa na tabela de dados. Feito isso clique em "Filter" topo lado direito da tabela para acessar os campos de filtrage do dados da tabela. Assim será possível pesquisar por uma palavra chave dentro da lista de notícias.

In [12]:
#@title Texto de título padrão
df

,Title,href
0,A força e os muitos amores da maior estrela do...,https://g1.globo.com/jornal-nacional/noticia/2...
1,'Mania de Você': os sucessos de Rita nas vozes...,https://g1.globo.com/jornal-nacional/noticia/2...
2,Álbum de 1973 de Rita com guitarrista Lucia Tu...,https://g1.globo.com/pop-arte/musica/blog/maur...
3,Desmatamento na Amazônia pode gerar perdas de ...,https://g1.globo.com/politica/noticia/2023/05/...
4,Imposto de Renda: vendeu carro com lucro? Saib...,https://g1.globo.com/economia/imposto-de-renda...
...,...,...
3841,Chef Jacquin come cenoura com terra e surpreen...,https://www.band.uol.com.br/entretenimento/mas...
3842,Nhoque garante primeiro pin de Luma; confira a...,https://www.band.uol.com.br/entretenimento/mas...
3843,Declaração final de líderes da Cúpula da Amazô...,https://www.band.uol.com.br/noticias/declaraca...
3844,Novo PAC terá obras de transporte e investimen...,https://www.band.uol.com.br/noticias/jornal-da...


Trecho abaixo importa módulo e habilita visualização interativa de dados do Google Colab.

In [13]:
from google.colab import data_table
from vega_datasets import data

#Habilitando display interativos para visualização dos dados.
data_table.enable_dataframe_formatter()


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Tabela das notícias coletadas
df

# Consultando Notícia Por Palavra Chave

In [ ]:
# Organizando lista de links (html). Esta saída foi o html usado no blog wsricardo.blogspot.com quando da publicação
# do cartum sobre arcabouço fiscal. O dataframe 'df' foi filtrado pelo tópico 'arcabouço'.
dfa = df['Title'].str.lower()
dfa = df[ dfa.str.contains('bolsonaro' ) ]

for news_title, news_href in zip( dfa['Title'], dfa['href'] ):
  print(f"<li style='padding: 2px;'><a href='{news_href}'>{news_title}</a></li>")

In [ ]:
# dataframe filtrado por tópico
#dfa = dfa['Title'].str.lower()
#df[  dfa.str.contains('inelegível')]
dfa[ dfa['Title'].str.contains('inelegível')]

In [ ]:
for news_title, news_href in zip( dfa[ dfa['Title'].str.contains('inelegível')]['Title'], dfa[ dfa['Title'].str.contains('inelegível')]['href'] ):
  print(f"<li style='padding: 2px;'><a href='{news_href}'>{news_title}</a></li>")

In [ ]:
#from IPython.core.display import display, HTML
# IPython Docs https://ipython.readthedocs.io/en/stable/index.html
from IPython.display import display, HTML
display(HTML('<h3>As Principais Notícias (Extraidas por Palavra Chave)</h3><br>'))
for news_title, news_href in zip( dfa[ dfa['Title'].str.contains('inelegível')]['Title'], dfa[ dfa['Title'].str.contains('inelegível')]['href'] ):
  display(HTML(f"<li style='padding: 2px;'><a href='{news_href}'>{news_title}</a></li>"))

A linha abaixo com comando 'df.to_csv' salvará o conteúdo do arquivo em formato csv.

In [ ]:
# Linha abaixo para salvar os dados da tabela.
df.to_csv('news.csv', index=False)

Para mais informações visite links abaixo.


Criado por [WSRicardo](https://github.com/wsricardo)

Blog: https://wsricardo.blogspot.com/

Banco das NOticias: https://github.com/wsricardo/newsfeed